In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../part_02/Jobs/full_jobs_df.csv')
list_of_skills = pd.read_csv('../part_02/Jobs/list_of_skills.csv')

In [3]:
df = df.dropna(subset = ['salary'])

In [4]:
df.reset_index(drop = True,inplace=True)

In [5]:
df.head()

,avg_salary,body,company,location,salary,title
0,123.30,the digital marketing director will develop an...,Splits59,"New York, NY",$85.6k–161k,Digital Marketing Director
1,128.95,job description\n\nreuters news agency is seek...,Thomson Reuters,"New York, NY",$85.9k–172k,"Senior Manager, Global Digital Marketing"
2,84.45,love data and conversions? come join our team....,CB Insights,"New York, NY",$57.9k–111k,Digital Marketing Manager
3,149.30,the head of digital marketing strategy and ope...,Broadridge,"New York, NY",$43.6k–255k,Head of Digital Marketing Strategy and Operations
4,84.45,about the role:\n\ndig is seeking a data-drive...,Dig Inn Support,"New York, NY",$57.9k–111k,Digital Marketing Manager


In [6]:
list_of_skills.head()

,0
0,Asking thoughtful questions
1,Data analysis
2,Information seeking
3,Interpretation
4,Judgment


In [7]:
df.body = [str(body).replace('\n',' ') for body in df.body]

In [8]:
def avg(i):
    if type(i) != float:
        try:
            average = (float(i.replace('k','').replace('M','').replace('$','').split('–')[0])+float(i.replace('k','').replace('M','').replace('$','').split('–')[1]))/2
        except:
            average = float(i.replace('k','').replace('$','').replace('M',''))
        return average
    else:
        return str(np.nan)

In [9]:
df['avg_salary'] = [avg(i) for i in df.salary]
df['avg_salary'] = df['avg_salary']*1000

In [10]:
df.sort_values('avg_salary',ascending=False).head()


,avg_salary,body,company,location,salary,title
300,315000.0,who we are cityblock health is a new type of h...,Cityblock Health,"Brooklyn, NY",$315k–315k,Staff Data Engineer
320,315000.0,namely’s mission is to help mid-sized companie...,Namely,"New York, NY",$315k–315k,Staff Data Engineer
483,313000.0,microsoft research ai (msr ai) seeks exception...,Microsoft,"Redmond, WA",$205k–421k,Researcher - Deep Learning
2004,289000.0,"founded in 2008, airbnb’s mission is to create...",Airbnb,"San Francisco, CA",$220k–358k,"Senior Manager, Airbnb Platform - New Products"
2160,286000.0,job title : director of analytics/ business in...,Milestone Inc,"Santa Clara, CA",$176k–396k,Director of Analytics/ Business intelligence


In [11]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split

In [12]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [13]:
my_list_of_stop_words = {'company','location','title'}

In [14]:
def add_stop_words(word):
    my_list_of_stop_words.add(word)

In [15]:
for i in ENGLISH_STOP_WORDS:
    add_stop_words(i)

In [20]:
for i in ['ability', 'able', 'advanced', 'age','applicants', 'application', 'applications', 'apply','based', 'benefits', 'best', 'big',
'candidate','color', 'com','day','disability','employees', 'employer', 'employment','equal', 'equal opportunity',
'equal opportunity employer','equivalent','gender', 'gender identity','include', 'including', 'job', 'join','large',
'level', 'like','ll','looking','members','opportunity employer','orientation','qualifications','race','religion','sex',
'sexual', 'sexual orientation','veteran', 'veteran status','york','company','location','national','origin','identity', 'opportunity', 'status']:
    add_stop_words(i)

In [21]:
cvec = CountVectorizer(stop_words = my_list_of_stop_words,ngram_range=(1,5))
cvec.fit(list_of_skills['0'])
vectors = cvec.transform(df['body'])

In [22]:
word_df = pd.DataFrame(data = vectors.todense(), columns = cvec.get_feature_names())

In [23]:
word_df.head()

,absence,absence timing,absence timing vacation,abstract,abstract connections,access,access methods,access outlook,access requests,account,...,writing focus,writing sales,writing sales communications,writing skills,written,written communication,written communications,youtube,youtube advertising,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
X = word_df
y=df.avg_salary

In [25]:
X_train, X_test,y_train,y_test = train_test_split(X,y)

In [26]:
lr = LinearRegression()

In [27]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [28]:
len(lr.coef_)

3980

In [29]:
import matplotlib.pyplot as plt

In [30]:
len(lr.coef_)

3980

In [31]:
lis = []
for i,j in zip(lr.coef_,word_df.columns):
    dic = {}
    dic['coef'] = i
    lis.append(dic)
pd.DataFrame(lis).set_index(word_df.columns).sort_values('coef',ascending=False)

,coef
abstract connections,1.226740e+16
action result structure stories,3.750078e+15
achievements individual,3.255690e+15
active listening,1.618744e+15
absence timing,1.317988e+15
action result,9.953970e+14
account teams campaigns,9.727683e+14
absence timing vacation,8.563697e+14
analyze recommend database improvements,4.170273e+14
agitated customers,2.542947e+14


In [510]:
pd.DataFrame(data = lr.coef_,columns = [i for i in full_df.columns]) #.T.sort_values(0,ascending=False)

ValueError: Shape of passed values is (147, 1), indices imply (147, 254)

In [181]:
lr.predict(X_test)

array([0., 0., 0., ..., 0., 0., 0.])

In [193]:
company_list = {}
for i,j in enumerate(df.company.unique()):
    company_list[i] = [j]

In [197]:
pd.DataFrame.from_dict(company_list).to_csv('./Company_List.csv')